In [1]:
import joblib
import numpy as np 
from pathlib import Path
import uproot 

In [2]:
syst_identifier = 'JESJER'
slurm_log_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/NewWorkflow/systematics_submit/{syst_identifier}/slurm_logs/'
slurm_sub_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/NewWorkflow/systematics_submit/{syst_identifier}/slurm_subs/'

wrapper_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/NewWorkflow/systematics_submit/{syst_identifier}/make_histogram_{syst_identifier}.sh'


In [3]:
for _path in [slurm_sub_path, slurm_log_path]:
    if not Path(_path).exists():
        Path(_path).mkdir(parents=True)

In [4]:
def generate_sub_header(i):
    script= '#!/bin/bash\n\n'+\
            '#SBATCH -C cpu\n'+\
            '#SBATCH -t 1:00:00\n'+\
            f'#SBATCH -J qgtagging_{syst_identifier}{i}\n'+\
            '#SBATCH --qos=shared\n'+\
            '#SBATCH --ntasks=1\n'+\
            '#SBATCH --cpus-per-task=10\n'+\
            '#SBATCH --account=atlas\n'+\
            '#SBATCH --mem=120G\n'+\
            f'#SBATCH --output={slurm_log_path}/slurm-%j.out\n'+\
            f'#SBATCH --error={slurm_log_path}/slurm-%j.err\n\n\n'
    return script


In [3]:
pythia_file_path='/global/cfs/projectdirs/atlas/hrzhao/qgcal/Samples_Dec11/pythia/pythiaA/user.wasu.Dec11a.mc16_13TeV.364709.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ9WithSW_minitrees.root/user.wasu.31564717._000001.minitrees.root'
pythia_file_path= Path(pythia_file_path)
pythia_root = uproot.open(pythia_file_path)

In [4]:
keys = [*pythia_root.keys()]

In [5]:
# remove the cycle number ';1'
keys = [key[:-2] for key in keys]

# remove the nominal tree
if 'nominal' in keys:
    keys.remove('nominal')

# Remove the following two tree 
#  'syst_PRW_DATASF__1down;1',
#  'syst_PRW_DATASF__1up;1']
keys = [key for key in keys if key.startswith('syst_JET')]


In [8]:
for i, block in enumerate(keys):
    with open(f'{slurm_sub_path}/{syst_identifier}_slurm_sub{i:03}.sl', 'w') as f:
        f.write(generate_sub_header(block))

        f.write(f"{wrapper_path} {block}\n")


In [6]:
keys.index('syst_JET_EffectiveNP_Statistical6__1up')

42